### Necessary Imports

In [1]:
import sys 
import pandas as pd
from api.assets import get_ticker_historical_data
## https://stackoverflow.com/questions/12257747/permanently-adding-a-file-path-to-sys-path-in-python
sys.path.append("/Users/lohyikuang/Downloads/personal_projects/the_heron_project/audrii/kedro-environment/env/kedro-environment/lib/python3.7/site-packages")

### Simulate data

In [2]:
simulated_data = {
    "GPS": [{"_id": "1", "action": "Long", "units": "20", "date": "2022-02-03", "price": "14.57", "simulatedFee": ""},
            {"_id": "2", "action": "Short", "units": "20", "date": "2022-04-04", "price": "14.21", "simulatedFee": ""},
            {"_id": "3", "action": "Long", "units": "25", "date": "2022-06-13", "price": "9.03", "simulatedFee": "0.2"}],
    "IAU": [{"_id": "1", "action": "Long", "units": "10", "date": "2022-03-02", "price": "36.61", "simulatedFee": ""},
            {"_id": "2", "action": "Short", "units": "10", "date": "2022-03-08", "price": "38.98", "simulatedFee": ""},
            {"_id": "3", "action": "Long", "units": "5", "date": "2022-03-15", "price": "36.45", "simulatedFee": "0.1"},
            {"_id": "4", "action": "Long", "units": "5", "date": "2022-04-13", "price": "37.62", "simulatedFee": "0.23"}],
    "M": [{"_id": "1", "action": "Long", "units": "55", "date": "2022-05-24", "price": "17.61", "simulatedFee": ""}],
    "NFLX": [{"_id": "1", "action": "Long", "units": "2", "date": "2022-01-19", "price": "515.86", "simulatedFee": ""}],
    "STNG": [{"_id": "1", "action": "Long", "units": "50", "date": "2022-04-24", "price": "20.60", "simulatedFee": "0.26"},
             {"_id": "2", "action": "Long", "units": "20", "date": "2022-05-13", "price": "28.35", "simulatedFee": "0.29"},
             {"_id": "3", "action": "Short", "units": "45", "date": "2022-05-24", "price": "30.46", "simulatedFee": "0.31"},
             {"_id": "4", "action": "Long", "units": "20", "date": "2022-06-02", "price": "35.19", "simulatedFee": "0.35"}]
}

### Get data for simulation

In [6]:
catalog.load("sample_GPS")

2022-06-20 23:40:56,514 - kedro.io.data_catalog - INFO - Loading data from `sample_GPS` (ParquetDataSet)...


,close,high,open,low,date,volume,symbol
0,31.61,32.2200,31.70,31.425,1624233600,7211571,GPS
1,32.29,32.3900,31.69,31.150,1624320000,4389757,GPS
2,32.94,33.2450,32.43,32.160,1624406400,3683844,GPS
3,33.10,33.3300,33.06,32.770,1624492800,3621631,GPS
4,33.99,34.2250,33.50,33.350,1624579200,8596141,GPS
...,...,...,...,...,...,...,...
247,9.03,9.3700,9.14,8.750,1655078400,13325406,GPS
248,9.12,9.2488,9.01,8.970,1655164800,9238146,GPS
249,9.17,9.4374,9.27,9.110,1655251200,11325980,GPS
250,8.89,9.0700,8.90,8.740,1655337600,11817111,GPS


In [34]:
tickers = ["GPS", "IAU", "M", "NFLX"]

raw_ticker_data = {}
for ticker in tickers:
    raw_ticker_data[ticker] = catalog.load(f"sample_{ticker}").set_index("date")

datasets = {}
for columnar in ['close', 'high', 'open', 'low', 'volume']: 
    dataset = pd.concat([
        raw_ticker_data[ticker][[columnar]].rename(columns = {columnar: ticker}) for ticker in tickers
    ], axis = 1).reset_index()
    dataset['datetime'] = dataset.date.apply(lambda x: pd.to_datetime(x, unit = "s"))

    assert not dataset.isnull().values.any()
    
    datasets[columnar] = dataset

2022-06-21 00:10:11,177 - kedro.io.data_catalog - INFO - Loading data from `sample_GPS` (ParquetDataSet)...
2022-06-21 00:10:11,188 - kedro.io.data_catalog - INFO - Loading data from `sample_IAU` (ParquetDataSet)...
2022-06-21 00:10:11,195 - kedro.io.data_catalog - INFO - Loading data from `sample_M` (ParquetDataSet)...
2022-06-21 00:10:11,203 - kedro.io.data_catalog - INFO - Loading data from `sample_NFLX` (ParquetDataSet)...


In [31]:
datasets['close']

,date,GPS,IAU,M,NFLX
0,1624233600,31.61,33.97,18.18,497.00
1,1624320000,32.29,33.83,18.65,508.82
2,1624406400,32.94,33.80,18.78,512.74
3,1624492800,33.10,33.77,19.45,518.06
4,1624579200,33.99,33.88,19.62,527.07
...,...,...,...,...,...
247,1655078400,9.03,34.62,21.22,169.69
248,1655164800,9.12,34.34,21.80,167.54
249,1655251200,9.17,34.80,21.92,180.11
250,1655337600,8.89,35.19,19.81,173.35


### Get our universe of symbols

In [ ]:
from api.mongodb import mongodb_client
tickers = mongodb_client.get_all_items("ticker_symbols")

In [ ]:
tickers = pd.DataFrame(tickers)
equity = tickers[tickers.quote_type == "Equity"]
etfs = tickers[tickers.quote_type == "ETF"]

### Test AlphaVantage

In [126]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [154]:
from urllib.request import Request, urlopen
from fake_useragent import UserAgent
import random
from bs4 import BeautifulSoup
from IPython.core.display import clear_output

# Here I provide some proxies for not getting caught while scraping
ua = UserAgent() # From here we generate a random user agent
proxies = [] # Will contain proxies [ip, port]

# Main function
def main():
    # Retrieve latest proxies
    proxies_req = Request('https://www.sslproxies.org/')
    proxies_req.add_header('User-Agent', ua.random)
    proxies_doc = urlopen(proxies_req).read().decode('utf8')

    soup = BeautifulSoup(proxies_doc, 'html.parser')
    proxies_table = soup.find(id='proxylisttable')
    print(soup)

    # Save proxies in the array
    for row in proxies_table.tbody.find_all('tr'):
        proxies.append({
          'ip':   row.find_all('td')[0].string,
          'port': row.find_all('td')[1].string
        })

    # Choose a random proxy
    proxy_index = random_proxy()
    proxy = proxies[proxy_index]

    for n in range(1, 20):
        req = Request('http://icanhazip.com')
        req.set_proxy(proxy['ip'] + ':' + proxy['port'], 'http')

        # Every 10 requests, generate a new proxy
        if n % 10 == 0:
            proxy_index = random_proxy()
            proxy = proxies[proxy_index]

        # Make the call
        try:
            my_ip = urlopen(req).read().decode('utf8')
            print('#' + str(n) + ': ' + my_ip)
            clear_output(wait = True)
        except: # If error, delete this proxy and find another one
            del proxies[proxy_index]
            print('Proxy ' + proxy['ip'] + ':' + proxy['port'] + ' deleted.')
            proxy_index = random_proxy()
            proxy = proxies[proxy_index]

# Retrieve a random index proxy (we need the index to delete it if not working)
def random_proxy():
    return random.randint(0, len(proxies) - 1)

In [175]:
proxies_0 = {
    "ip": "213.230.120.185",
    "port": 1500,
}

proxies_1 = {
    "ip": "81.4.102.223",
    "port": 8081,
}
user_agent_0 = 'Mozilla/5.0 (Windows NT 6.3; WOW64; Trident/7.0; rv:11.0) like Gecko'
user_agent_1 = 'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Trident/5.0)'

headers_0 = {'User-Agent': user_agent_0, "Accept-Language": "en-US, en;q=0.5"}
headers_1 = {'User-Agent': user_agent_1, "Accept-Language": "en-US, en;q=0.5"}

In [191]:
from audrii.utilities.scraper.tickers.alpha_vantage import get_historical_data

d = get_historical_data("K8ROJ206V3NNEBFO","GPS", data_format="csv", proxy = proxies_0, header = headers_0)
d

{
    "Note": "Thank you for using Alpha Vantage! Our standard API call frequency is 5 calls per minute and 500 calls per day. Please visit https://www.alphavantage.co/premium/ if you would like to target a higher API call frequency."
}


RateLimitException: API call frequency limit hit.